In [1]:
import pandas as pd
from cassandra.cluster import Cluster

In [2]:
file_name = "Template Data Samples.xlsx"

# Create a dictionary having each sheet as a dataframe
xl_job_meta_data = pd.read_excel(file_name, sheet_name=None)

In [3]:
type(xl_job_meta_data)

dict

In [4]:
job_type = xl_job_meta_data["Job_Type"]
type(job_type)

pandas.core.frame.DataFrame

In [5]:
# "Software_Engineer", "Data_Analyst", "Systems_Engineer"
type(job_type["Data_Analyst"])

pandas.core.series.Series

In [6]:
#job_type = job_type.replace(np.nan, '', regex=True)
job_type.fillna('', inplace=True)

In [7]:
job_type["Data_Analyst"]

0           accounting
1              airflow
2                  aws
3      azure data lake
4             big data
            ...       
131                   
132                   
133                   
134                   
135                   
Name: Data_Analyst, Length: 136, dtype: object

In [8]:
# to filter out NaN values at a specific column >> df.query("col_name == col2")
# https://stackoverflow.com/questions/43821529/filter-out-nan-rows-in-a-specific-column
software_engineer_keywords = job_type.query("Software_Engineer == Software_Engineer")["Software_Engineer"]
data_analyst_keywords = job_type.query("Data_Analyst == Data_Analyst")["Data_Analyst"]
systems_engineer_keywords = job_type.query("Systems_Engineer == Systems_Engineer")["Systems_Engineer"]

In [9]:
for item in job_type:
    print(item)

Idx
Software_Engineer
Data_Analyst
Systems_Engineer


In [11]:
#from cassandra.cluster import Cluster
#cluster = Cluster(ip_address)
cluster = Cluster(['127.0.0.1'])
#session = cluster.connect(keyspace_name)
session = cluster.connect("jobsearch_keyspace")
query = "INSERT INTO job_type(idx, software_engineer, data_analyst, systems_engineer) VALUES (?,?,?,?)"
prepared = session.prepare(query)

In [12]:
for item in job_type:
    print(job_type[item][0], job_type[item][1], job_type[item][2], job_type[item][3])

1 2 3 4
ad tech ajax algorithm android
accounting airflow aws azure data lake
2g 3com 3g 4g lte


In [13]:
print(job_type['Idx'][0], job_type['Software_Engineer'][0], job_type['Data_Analyst'][0], job_type['Systems_Engineer'][0])

1 ad tech accounting 2g


In [14]:
job_type['Idx'].count()

136

In [15]:
idx = 0
total_features = job_type['Idx'].count()
while idx < total_features:
    session.execute(prepared, (job_type['Idx'][idx], job_type['Software_Engineer'][idx], 
                               job_type['Data_Analyst'][idx], job_type['Systems_Engineer'][idx]))
    idx += 1

In [16]:
# read from Cassandra to dataframe
# order by doesn't work so sort_values by is used later
select_query = "SELECT * from job_type"
df_job_type = pd.DataFrame(list(session.execute(select_query)))

In [17]:
# another way to read from Cassandra
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

select_query = "SELECT * from job_type"
rslt = session.execute(select_query, timeout=None)
df = rslt._current_rows

In [18]:
# order by idx column 
df_job_type.sort_values(by=['idx'])

,idx,data_analyst,software_engineer,systems_engineer
40,1,accounting,ad tech,2g
49,2,airflow,ajax,3com
134,3,aws,algorithm,3g
52,4,azure data lake,android,4g lte
12,5,big data,apache druid,5gnr
...,...,...,...,...
11,132,,xml,
124,133,,xml parser,
28,134,,xpath,
68,135,,xquery,
